# 추론: Inferring
이 수업에서는 제품 리뷰와 뉴스 기사에서 정서와 주제를 추론할 것입니다.

## 설정

In [ ]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

## 제품 리뷰 글

In [ ]:
lamp_review = """
내 침실에 쓸 멋진 램프가 필요했고, 이 램프는 추가 수납이 가능했고 가격도 너무 비싸지 않았다. 빨리 잡았어요.  우리 램프의 끈이 운송 중에 끊어져서 회사에서 기쁘게 새 램프를 보냈습니다. 며칠 안에 도착하기도 했어요. 그것은 조립하기 쉬웠다.  제가 부족한 부분이 있어서 지원팀에 연락을 해봤는데, 그들은 아주 빨리 부족한 부분을 저에게 가져다 주었어요! 루미나는 내가 보기에 그들의 고객과 제품을 신경쓰는 훌륭한 회사인 것 같아요!!
"""

## 정서 (긍정/부정)

In [ ]:
prompt = f"""
삼중 백틱으로 구분되는 다음 제품 리뷰는 어떤 느낌인가요?

리뷰 텍스트:'''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

In [ ]:
prompt = f"""
삼중 백틱으로 구분되는 다음 제품 리뷰는 어떤 느낌인가요?

"긍정적" 또는 "부정적" 중 하나로 대답하십시오.

리뷰 텍스트:'''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

## 감정 유형 식별

In [ ]:
prompt = f"""
다음 리뷰의 작성자가 표현하고 있는 감정 목록을 확인합니다. 
목록에 5개 이하의 항목을 포함합니다. 
답변 형식을 쉼표로 구분된 소문자 단어 목록으로 지정합니다.

리뷰 텍스트:'''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

## 분노 식별

In [ ]:
prompt = f"""
다음 리뷰의 작가는 분노를 표현하고 있나요? 
리뷰는 삼중 백틱으로 구분된다. 
예 또는 아니오 중 하나로 대답하십시오.

리뷰 텍스트:'''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

## 고객 리뷰에서 제품 및 회사 이름 추출

In [ ]:
prompt = f"""
리뷰 텍스트에서 다음 항목을 식별합니다: 
- 리뷰어가 구매한 품목
- 그 물건을 만든 회사

그 리뷰는 삼중 백틱으로 구분된다. 
응답을 "항목"과 "브랜드"를 키로 하여 JSON 개체로 형식을 지정합니다. 
정보가 없으면 "unknown"을 값으로 사용합니다.
가능한 한 짧게 응답하십시오.
  
리뷰 텍스트:'''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

## 한 번에 여러 작업하기

In [ ]:
prompt = f"""
리뷰 텍스트에서 다음 항목을 식별합니다: 
- 정서(긍정적 또는 부정적)
- 리뷰어가 분노를 표현하고 있습니까? (참 또는 거짓)
- 검토자가 구매한 품목
- 그 물건을 만든 회사

그 리뷰는 삼중 백틱으로 구분된다. 
"Sentment", "Anger", "Item", "Brand"를 키로 하여 응답을 JSON 개체로 포맷합니다.
정보가 없으면 "알 수 없음"을 값으로 사용합니다.
가능한 한 짧게 응답하십시오.
분노 값을 부울 형식으로 지정합니다.

리뷰 텍스트: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

## 주제 유추

In [ ]:
story = """
정부가 실시한 최근 조사에서, 공공 부문 직원들은 그들이 일하는 부서에 대한 만족도를 평가하도록 요청받았다. 결과는 NASA가 95%의 만족도를 기록하며 가장 인기 있는 부서라는 것을 보여주었다.

NASA의 한 직원인 John Smith는 이 연구 결과에 대해 "나는 나사가 1위를 차지한 것에 놀라지 않았다. 이곳은 놀라운 사람들과 놀라운 기회들과 함께 일하기에 좋은 곳입니다. 저는 이런 혁신적인 조직의 일원이 된 것이 자랑스럽습니다."

이 결과는 NASA의 경영진에게도 환영을 받았으며, 톰 존슨 소장은 "우리 직원들이 나사에서 일하는 것에 만족한다는 말을 듣게 되어 매우 기쁩니다. 우리는 우리의 목표를 달성하기 위해 끊임없이 노력하는 재능 있고 헌신적인 팀이 있는데, 그들의 노력이 성과를 내고 있다는 것을 보는 것은 환상적이다."

이번 조사에서도 사회보장청의 만족도가 가장 낮은 것으로 나타나 직원의 45%만이 직무에 만족한다고 응답했다. 정부는 설문조사에서 직원들이 제기한 우려를 해소하고 전 부서에 걸쳐 직무 만족도를 향상시키기 위해 노력할 것을 약속했다."""

## 주제 5개 유추하기

In [ ]:
prompt = f"""
삼중 백틱으로 구분되는 다음 텍스트에서 다뤄지는 주제 다섯 가지를 결정합니다. 

각 항목을 한 단어 또는 두 단어 길이로 만드십시오. 

응답 형식을 쉼표로 구분된 항목 목록으로 지정합니다.

텍스트 예시: '''{story}'''
"""
response = get_completion(prompt)
print(response)

In [ ]:
response.split(sep=',')

In [ ]:
topic_list = [
    "나사", "정부", "공학", "직원 만족", "연방 정부"
]

## 특정 항목에 대한 뉴스 알림 만들기

In [ ]:
prompt = f"""
다음 텍스트의 주제 목록 내 항목 포함 여부를 판단하세요.
텍스트는 삼중 백틱으로 구분됩니다.

각 주제에 대해 0 또는 1로 답변을 목록으로 제공합니다.

주제 목록: {", ".join(topic_list)}

텍스트 예시: '''{story}'''
"""
response = get_completion(prompt)
print(response)

In [ ]:
topic_dict = {i.split(': ')[0].strip('- '): int(i.split(': ')[1]) for i in response.split(sep='\n')}
print(topic_dict)
if topic_dict['나사'] == 1:
    print("경고: 나사의 새로운 이야기!")

## 스스로 실험해 보세요!